# Interactive TIFF Overlay Viewer
Quickly visualize alignment between raw images and labels.
- Sourced from label folder
- Assumes identical filenames in image folder

In [1]:
import tifffile
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from ipywidgets import interact, IntSlider
import ipywidgets as widgets

# Configure Paths
# (Assuming running from sam_trainer/notebooks/, adjust .. as needed)
image_folder = Path(r"C:\Repos\sabine\test_data")
label_folder = Path(r"C:\Repos\sabine\test_inf")

print(f"Images: {image_folder.resolve()}")
print(f"Labels: {label_folder.resolve()}")

Images: C:\Repos\sabine\test_data
Labels: C:\Repos\sabine\test_inf


In [2]:
def get_files(folder, pattern="*.tif"):
    return sorted(list(folder.glob(pattern)))


label_files = get_files(label_folder)
print(f"Found {len(label_files)} label files.")


def view_overlay(index):
    if not label_files:
        print("No files to show.")
        return

    label_path = label_files[index]

    # Corresponding image path: handle _masks suffix
    # e.g. "image001_masks.tif" -> "image001.tif"
    image_name = label_path.name.replace("_masks.tif", ".tif")
    image_path = image_folder / image_name

    if not image_path.exists():
        print(f"Image missing: {image_path}")
        # Try fallbacks?
        return

    # Load Data
    try:
        lbl = tifffile.imread(label_path)
        img = tifffile.imread(image_path)
    except Exception as e:
        print(f"Error loading files: {e}")
        return

    # Handle 3D stacks - take middle slice for speed if needed, or first
    # Keeping it simple: if 3D, take index 0
    if img.ndim == 3:
        img = img[0]
    if lbl.ndim == 3:
        lbl = lbl[0]

    # Plotting
    fig, ax = plt.subplots(figsize=(10, 10))

    # Raw image
    ax.imshow(img, cmap="gray")

    # Label overlay (masked)
    masked_lbl = np.ma.masked_where(lbl == 0, lbl)
    ax.imshow(masked_lbl, cmap="jet", alpha=0.2, interpolation="none")

    ax.set_title(
        f"[{index}] {image_name}\nImg: {img.shape} {img.dtype} | Lbl: {lbl.shape}"
    )
    plt.axis("off")
    plt.show()


if label_files:
    interact(
        view_overlay,
        index=IntSlider(
            min=0, max=len(label_files) - 1, step=1, value=0, description="File Idx"
        ),
    )
else:
    print("No label files found. Please check the paths in the cell above.")

Found 96 label files.


interactive(children=(IntSlider(value=0, description='File Idx', max=95), Output()), _dom_classes=('widget-int…